In [ ]:
import csv, os, shutil, datetime, sys, pyodbc, decimal, subprocess, re, json
from pathlib import Path

In [2]:
current_dateTime = datetime.datetime.today().strftime('%Y-%m-%d') # actual datetime_ to be added into filename for archiving
folder = Path('C:/Users/***/csv') # set to variable later
Path(f'{folder}/Archive').mkdir(parents=True, exist_ok=True)
folder_arch = Path('C:/Users/***/Archive/') #set to variable later
folder_SQL_Scripts = Path('C:/Users/***/SQL_Scripts/') #set to variable later
sql_file = list(folder_SQL_Scripts.glob('*TEST.sql'))


In [ ]:
# data reference connection as well as final destination for data load 


with open('C:/Users/***/SQL_conn_settings.json', 'r') as file:
    data = json.load(file)

conn_0 = pyodbc.connect(f"Driver={data['Driver']};" 
                      f"Server={data['Server']};"
                      f"Database={data['Database']};"
                      f"Trusted_Connection={data['Trusted_Connection']};")


In [ ]:
CSV_file = list(folder.glob('*.csv')) # very important thing - if .glob used without list - it will contain a path value untill first use, after` that it is erased - this caused my archiving to break
folder_arch_A = folder_arch.glob('*.csv')
print(CSV_file) # debug line

In [ ]:
for f in CSV_file:
    CSV_file_name = os.path.basename(f)
    print(CSV_file_name) #debug line


WEEKLY_NEW.csv


In [6]:
#in case of bulk load, could stay for later to iterate through files - filename to be filtered as 2 files are being loaded per iteration
for f in sql_file:
    sql_path = f
    sql_f = os.path.basename(f)
    print(sql_f) #debug line

Weekly_load_integrated_TEST.sql


In [ ]:
# executing powershell loader with subprocess library 
ps_script = r"C:\Users\***\CL1.ps1" # change to relative path later
function = "W" # change to variable for scaling later
# params add here
cmd = [
    "powershell.exe",
    "-ExecutionPolicy", "Bypass",
    "-Command",
    f'. "{ps_script}"; {function}'

]

# run
result = subprocess.run(cmd, capture_output=True, text=True)

# read shell reply
print("=== STDOUT ===")
print(result.stdout)
print("=== STDERR ===")
print(result.stderr)
print("Exit code:", result.returncode)


=== STDOUT ===
WEEKLY DATA LOADED
WEEKLY DATA TRANSFORMED
WEEKLY DATA LOAD EXECUTED

=== STDERR ===

Exit code: 0


In [ ]:
def DQ_check(conn, cur): # DQ query - have to adjust for every concrete csv ingested
#CHECKING NUMBER OF ROWS AND SUM OF VOL AND COMPARING AGAINST CLEAN DATA - DIFFERENCES MORE THAN 25% WILL RAISE ALERT#
#    with conn_0:
#        cur_0 = conn_0.cursor()

    sqlQuery = "SELECT SUM(cast(vol AS NUMERIC))/COUNT(DISTINCT(dt))/1000000000,COUNT(dt)/COUNT(DISTINCT(dt)), dt FROM [DB].[temp_temp] WHERE period = 'week' AND dt <(SELECT MAX(dt) FROM [DB].[temp_temp])  group by dt"
    sqlQuery_0 = "SELECT SUM(cast(vol AS NUMERIC))/COUNT(DISTINCT(dt))/1000000000,COUNT(dt)/COUNT(DISTINCT(dt)) FROM [DB].[master] WHERE period = 'week' and dt <(select max(dt) from [DB].[master]) and dt>= DATEADD(DAY, -90, GETDATE())"
    cur_0.execute(sqlQuery_0)
    K = cur_0.fetchall()
    
    for x,v in K: # Getting average meanings for num of rows and gross vol from benchmark DB
        print(f'Average Gross Volume: {x} Number of rows: {v}')
        #None - remove hash after debug 
    cur_0.execute(sqlQuery)
    L = cur_0.fetchall()
    for k,c,i in L:
        

        if ((decimal.Decimal(k)/x)-1)*100 >= 25:
            print(f'Gross Volume ({k}) is higher than average meaning ({x}) for more than 25%, please review date from {i}')
            print('Do you want to continue loading? y/n')
            while True:
                answer = input()
                if answer == 'y':
                    return(answer)
                    break
                if answer == 'n':
                    return(answer)
                    break
                else:
                    print('Please provide correct answer, y/n')

        elif ((float(c)/v)-1)*100 >= 25:
            print(f'Number of rows ({k}) is higher than average meaning ({v}) for more than 25%, please review date from {i}')
            print('Do you want to continue loading? y/n')
            while True:
                answer = input()
                if answer == 'y':
                    return(answer)
                    break
                if answer == 'n':
                    return(answer)
                    break
                else:
                    print('Please provide correct answer, y/n')

        elif ((decimal.Decimal(k)/x)-1)*100 <= -25:
            print(f'Gross Volume ({k}) is lower than average meaning ({x}) for more than 25%, please review date from {i}')
            print('Do you want to continue loading? y/n')
            while True:
                answer = input()
                if answer == 'y':
                    return(answer)
                    break
                if answer == 'n':
                    return(answer)
                    break
                else:
                    print('Please provide correct answer, y/n')

        elif ((float(c)/v)-1)*100 <= -25:
            print(f'Number of rows ({k}) is lower than average meaning ({v}) for more than 25%, please review date from {i}')
            print('Do you want to continue loading? y/n')
            while True:
                answer = input()
                if answer == 'y':
                    return(answer)
                    break
                if answer == 'n':
                    return(answer)
                    break
                else:
                    print('Please provide correct answer, y/n')

        else:
            print(f'Date: {i} Gross Volume: {k} Number of rows: {c}')
    print('Do you want to continue loading? y/n')
    while True:
        answer = input()
        if answer == 'y':
            return(answer)
            break
        if answer == 'n':
            return(answer)
            break
        else:
            print('Please provide correct answer, y/n')



In [9]:
def exec_sql_file(cur_0, sql_file):
    #SCRIPT TO RUN SQL FILE 
    print ("\n[INFO] Executing SQL script file: '%s'" % (sql_file))
    statement = ""
    

    
    for line in open(sql_file,'r',encoding='utf-8'):

        print(line)
        
        if re.match(r'--', line):  # ignore sql comment lines
            continue
        if not re.search(r';$', line):  # keep appending lines that don't end in ';'
            statement = statement + line
            
        else:  # when you get a line ending in ';' then exec statement and reset for next statement
            statement = statement + line
            
            print ("\n\n[DEBUG] Executing SQL statement:\n%s" % (statement))
            try:
                with conn_0:
                    cur_0 = conn_0.cursor()
                    cur_0.execute(statement)
                    conn_0.commit()
            except (OperationalError, ProgrammingError) as e:
                print("\n[WARN] MySQLError during execute statement \n\tArgs: '%s'" % (str(e.args)))

            statement = ""
    

In [10]:
with conn_0:
    cur_0 = conn_0.cursor()
    repl = DQ_check(conn_0, cur_0)

Average Gross Volume: 56.049036 Number of rows: 31345
Date: 2025-08-04 00:00:00 Gross Volume: 57.857225 Number of rows: 31695
Date: 2025-08-11 00:00:00 Gross Volume: 59.463271 Number of rows: 31458
Date: 2025-08-18 00:00:00 Gross Volume: 57.441074 Number of rows: 31843
Date: 2025-08-25 00:00:00 Gross Volume: 56.030912 Number of rows: 32053
Date: 2025-09-01 00:00:00 Gross Volume: 56.245135 Number of rows: 31470
Date: 2025-09-08 00:00:00 Gross Volume: 56.497257 Number of rows: 31123
Date: 2025-09-15 00:00:00 Gross Volume: 54.857882 Number of rows: 30742
Date: 2025-09-22 00:00:00 Gross Volume: 53.609987 Number of rows: 30717
Date: 2025-09-29 00:00:00 Gross Volume: 51.950150 Number of rows: 30851
Do you want to continue loading? y/n


 y


In [ ]:
if repl == 'n':

    try:
        sys.exit()
    except:
        print('Exiting')

elif repl == 'y':
    with conn_0:
        cur_0 = conn_0.cursor()
        print('Loading started')
        cur_0.execute("INSERT INTO [DB].[table_new] SELECT * FROM [DB].[temp_temp]")
        cur_0.execute('DELETE from [DB].[temp_temp]')
        
    
        print('Data loaded, archiving files')
        for f in list(CSV_file): # moving file into archive folder
            CSV_file_name = os.path.basename(f)
            print(CSV_file_name)
            shutil.move(folder/CSV_file_name,folder_arch/CSV_file_name)
    
    
        for F in list(folder_arch_A): #renaming in the archived folder
            CSV_file_name = os.path.basename(F)
            CSV_file_name_F = str(CSV_file_name[:-4])
            archiveName = str(CSV_file_name_F)+'_'+ str(current_dateTime+'.csv')
            shutil.move(folder_arch/F,folder_arch/archiveName)
        print('Archivisation complete')





Loading started
Data loaded, archiving files
WEEKLY_NEW.csv
Archivisation complete


In [ ]:
exec_sql_file(cur_0,sql_path)

cur_0.close()
conn_0.close()